In [46]:
import csv
from os import listdir
from os.path import isfile, join,getsize
import glob
import time
import random
from multiprocessing import Pool
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')
#from scipy.interpolate import interp1d
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import pickle as pkl
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import dask.array as da
#from sklearn.decomposition import PCA,TruncatedSVD,NMF
import dask.dataframe as dd
import warnings
warnings.simplefilter(action='ignore', category=ResourceWarning)
np.random.seed(seed=1991)
import feather
import tables

In [47]:
modifiers_list=pd.read_csv("/data/dharp/compounding/datasets/modifiers_reduced.csv",delimiter="\t",usecols =[0])
modifiers_list=modifiers_list.modifier.unique()
modifiers_list.shape

(54400,)

In [48]:
heads_list=pd.read_csv("/data/dharp/compounding/datasets/heads_reduced.csv",delimiter="\t",usecols =[0])
heads_list=heads_list['head'].unique()
heads_list.shape

(53466,)

In [49]:
words_list=np.union1d(modifiers_list, heads_list)
words_list.shape

(83254,)

In [50]:
word_list=[]
for word in words_list:
    word_list.append(word+"oun")
len(word_list)

83254

In [51]:
input_files = [f for f in glob.glob("/data/dharp/compounding/datasets/entire_df_*_.h5") if isfile(join("/data/dharp/compounding/datasets/", f))]
files=[]
for f in input_files:
    files.append(f)
np.random.shuffle(files)

In [52]:
br_to_us=pd.read_excel("Book.xlsx")
br_to_us_dict=dict(zip(br_to_us.UK.tolist(),br_to_us.US.tolist()))

contextwords_df=pd.read_csv("contexts.csv",sep="\t")
contextwords=contextwords_df.context.tolist()


adv_dict=dict(zip(['adv'],['r']))
adv_replacement={'context_pos':adv_dict}
spelling_replacement={'context':br_to_us_dict,'modifier':br_to_us_dict,'head':br_to_us_dict,'word':br_to_us_dict}


pos_replacement={'pos':dict(zip(["noun","verb","adj"],['n','v','a']))}

In [53]:
#@da.as_gufunc(signature="(i)->()", output_dtypes=str, vectorize=True)
def lemma_maker(x, y):
    return lemmatizer.lemmatize(x,y)

In [54]:
def relemjoin(df,col_name,lemmatize=True):
    new_col=col_name.split('_')[0]
    new_col_pos=new_col[0]+"_pos"
    df[new_col]=df[col_name].str.split('_', 1).str[0]
    df[new_col_pos]="n"
    if lemmatize==True:
        df[new_col]=np.vectorize(lemma_maker)(df[new_col], df[new_col_pos])
    df[new_col]=df[new_col]+"_n"
    return df

In [55]:
def syntactic_reducer(df,align,level=None):
    if len(df) == 0:
        print("Am here")
        return df
    if align=="right":
        if level=="word":
            #t1=time.time()
            df=df.loc[df.fivegram_pos.str.match(r'^[-a-z]+_noun\s+[-a-z]+_.+\s+[-a-z]+_.+\s+[-a-z]+_.+\s+[-a-z]+_.+$')]
            if len(df) == 0:
                return df
            
            df['word_pos'],df['r1_pos'],df['r2_pos'],df['r3_pos'],_=df['fivegram_pos'].str.split(r'\s+').str
            #df=df.query('word_pos == @word_list')
            df=relemjoin(df,'word_pos')
            df=pd.melt(df,id_vars=['word','decade','count'],value_vars=['r1_pos','r2_pos','r3_pos'])
            #print(time.time()-t1)
            return df
        else:
            phrases=df.loc[df.fivegram_pos.str.match(r'^[-a-z]+_noun\s+[-a-z]+_noun\s+[-a-z]+_.+\s+[-a-z]+_.+\s+[-a-z]+_.+$')]
            cdsm=phrases.loc[~phrases.fivegram_pos.str.match(r'^[-a-z]+_noun\s+[-a-z]+_noun\s+[a-z-]+_noun\s+[a-z-]+_.+\s+[a-z-]+_.+$')]

            try:
                phrases['modifier_pos'],phrases['head_pos'],phrases['r1_pos'],phrases['r2_pos'],phrases['r3_pos']=phrases['fivegram_pos'].str.split(r'\s+').str
                cdsm['modifier_pos'],cdsm['head_pos'],cdsm['r1_pos'],cdsm['r2_pos'],cdsm['r3_pos']=cdsm['fivegram_pos'].str.split(r'\s+').str
            except ValueError:
                phrases=pd.DataFrame()
                compounds=pd.DataFrame()
                modifiers=pd.DataFrame()
                heads=pd.DataFrame()
                return phrases,compounds,modifiers,heads
            
            phrases=relemjoin(phrases,'modifier_pos',lemmatize=False)
            phrases=relemjoin(phrases,'head_pos')
            cdsm=relemjoin(cdsm,'modifier_pos',lemmatize=False)
            cdsm=relemjoin(cdsm,'head_pos')
            
            phrases=pd.melt(phrases,id_vars=['modifier','head','decade','count'],value_vars=['r1_pos','r2_pos','r3_pos'])
            compounds=pd.melt(cdsm,id_vars=['modifier','head','decade','count'],value_vars=['r1_pos','r2_pos','r3_pos'])
            modifiers=pd.melt(cdsm,id_vars=['modifier','decade','count'],value_vars=['head','r1_pos','r2_pos'])
            heads=pd.melt(cdsm,id_vars=['head','decade','count'],value_vars=['modifier','r1_pos','r2_pos','r3_pos'])

            return phrases,compounds,modifiers,heads
            
            
    elif align=="mid1":
        if level=="word":
            df=df.loc[df.fivegram_pos.str.match(r'^[a-z-]+_.+\s+[a-z-]+_noun\s+[a-z-]+_.+\s+[a-z-]+_.+\s+[a-z-]+_.+$')]
            if len(df) == 0:
                return df
            
            df['l1_pos'],df['word_pos'],df['r1_pos'],df['r2_pos'],df['r3_pos']=df['fivegram_pos'].str.split(r'\s+').str
            df=relemjoin(df,'word_pos')
            df=pd.melt(df,id_vars=['word','decade','count'],value_vars=['l1_pos','r1_pos','r2_pos','r3_pos'])
            return df
        else:
            phrases=df.loc[df.fivegram_pos.str.match(r'^[a-z-]+_.+\s+[a-z-]+_noun\s+[a-z-]+_noun\s+[a-z-]+_.+\s+[a-z-]+_.+$')]
            cdsm=phrases.loc[~phrases.fivegram_pos.str.match(r'^[a-z-]+_noun\s+[a-z-]+_noun\s+[a-z-]+_noun\s+[a-z-]+_noun\s+[a-z-]+_.+$')]

            try:
                phrases['l1_pos'],phrases['modifier_pos'],phrases['head_pos'],phrases['r1_pos'],phrases['r2_pos']=phrases['fivegram_pos'].str.split(r'\s+').str
                cdsm['l1_pos'],cdsm['modifier_pos'],cdsm['head_pos'],cdsm['r1_pos'],cdsm['r2_pos']=cdsm['fivegram_pos'].str.split(r'\s+').str
            except ValueError:
                phrases=pd.DataFrame()
                compounds=pd.DataFrame()
                modifiers=pd.DataFrame()
                heads=pd.DataFrame()
                return phrases,compounds,modifiers,heads
            
            phrases=relemjoin(phrases,'modifier_pos',lemmatize=False)
            phrases=relemjoin(phrases,'head_pos')
            cdsm=relemjoin(cdsm,'modifier_pos',lemmatize=False)
            cdsm=relemjoin(cdsm,'head_pos')
            
            phrases=pd.melt(phrases,id_vars=['modifier','head','decade','count'],value_vars=['l1_pos','r1_pos','r2_pos'])
            compounds=pd.melt(cdsm,id_vars=['modifier','head','decade','count'],value_vars=['l1_pos','r1_pos','r2_pos'])
            modifiers=pd.melt(cdsm,id_vars=['modifier','decade','count'],value_vars=['head','l1_pos','r1_pos','r2_pos'])
            heads=pd.melt(cdsm,id_vars=['head','decade','count'],value_vars=['modifier','l1_pos','r1_pos','r2_pos'])
            return phrases,compounds,modifiers,heads
    
            
    elif align=="mid2":
        if level=="word":
            df=df.loc[df.fivegram_pos.str.match(r'^[a-z-]+_.+\s+[a-z-]+_.+\s+[a-z-]+_noun\s+[a-z-]+_.+\s+[a-z-]+_.+$')]
            if len(df) == 0:
                return df
            
            df['l1_pos'],df['l2_pos'],df['word_pos'],df['r1_pos'],df['r2_pos']=df['fivegram_pos'].str.split(r'\s+').str
            df=relemjoin(df,'word_pos')
            df=pd.melt(df,id_vars=['word','decade','count'],value_vars=['l1_pos','l2_pos','r1_pos','r2_pos'])
            return df
        else:
            
            phrases=df.loc[df.fivegram_pos.str.match(r'^[a-z-]+_.+\s+[a-z-]+_.+\s+[a-z-]+_noun\s+[a-z-]+_noun\s+[a-z-]+_.+$')]
            cdsm=phrases.loc[~phrases.fivegram_pos.str.match(r'^[a-z-]+_.+\s+[a-z-]+_noun\s+[a-z-]+_noun\s+[a-z-]+_noun\s+[a-z-]+_noun$')]

            try:
                phrases['l1_pos'],phrases['l2_pos'],phrases['modifier_pos'],phrases['head_pos'],phrases['r1_pos']=phrases['fivegram_pos'].str.split(r'\s+').str
                cdsm['l1_pos'],cdsm['l2_pos'],cdsm['modifier_pos'],cdsm['head_pos'],cdsm['r1_pos']=cdsm['fivegram_pos'].str.split(r'\s+').str
            except ValueError:
                phrases=pd.DataFrame()
                compounds=pd.DataFrame()
                modifiers=pd.DataFrame()
                heads=pd.DataFrame()
                return phrases,compounds,modifiers,heads
            
            phrases=relemjoin(phrases,'modifier_pos',lemmatize=False)
            phrases=relemjoin(phrases,'head_pos')
            cdsm=relemjoin(cdsm,'modifier_pos',lemmatize=False)
            cdsm=relemjoin(cdsm,'head_pos')
            
            phrases=pd.melt(phrases,id_vars=['modifier','head','decade','count'],value_vars=['l1_pos','l2_pos','r1_pos'])
            compounds=pd.melt(cdsm,id_vars=['modifier','head','decade','count'],value_vars=['l1_pos','l2_pos','r1_pos'])
            modifiers=pd.melt(cdsm,id_vars=['modifier','decade','count'],value_vars=['head','l1_pos','l2_pos','r1_pos'])
            heads=pd.melt(cdsm,id_vars=['head','decade','count'],value_vars=['modifier','l1_pos','l2_pos','r1_pos'])
            return phrases,compounds,modifiers,heads
            
            
    elif align=="mid3":
        df=df.loc[df.fivegram_pos.str.match(r'^[a-z-]+_.+\s+[a-z-]+_.+\s+[a-z-]+_.+\s+[a-z-]+_noun\s+[a-z-]+_.+$')]
        if len(df)==0:
            return df

        df['l1_pos'],df['l2_pos'],df['word_pos'],df['r1_pos'],df['r2_pos']=df['fivegram_pos'].str.split(r'\s+').str
        df=relemjoin(df,'word_pos')
        df=pd.melt(df,id_vars=['word','decade','count'],value_vars=['l1_pos','l2_pos','r1_pos','r2_pos'])
        return df
        
    elif align=="left":
        
        if level=="word":
            df=df.loc[df.fivegram_pos.str.match(r'^[a-z-]+_.+\s+[a-z-]+_.+\s+[a-z-]+_.+\s+[a-z-]+_.+\s+[a-z-]+_noun$')]
            if len(df) == 0:
                return df
            _,df['l1_pos'],df['l2_pos'],df['l3_pos'],df['word_pos']=df['fivegram_pos'].str.split(r'\s+').str
            df=relemjoin(df,'word_pos')
            df=pd.melt(df,id_vars=['word','decade','count'],value_vars=['l1_pos','l2_pos','l3_pos'])
            return df
        else:
            phrases=df.loc[df.fivegram_pos.str.match(r'^[a-z-]+_.+\s+[a-z-]+_.+\s+[a-z-]+_.+\s+[a-z-]+_noun\s+[a-z-]+_noun$')]
            cdsm=phrases.loc[~phrases.fivegram_pos.str.match(r'^[a-z-]+_.+\s+[a-z-]+_.+\s+[a-z-]+_noun\s+[a-z-]+_noun\s+[a-z-]+_noun$')]
            
            try:
                phrases['l1_pos'],phrases['l2_pos'],phrases['l3_pos'],phrases['modifier_pos'],phrases['head_pos']=phrases['fivegram_pos'].str.split(r'\s+').str
                cdsm['l1_pos'],cdsm['l2_pos'],cdsm['l3_pos'],cdsm['modifier_pos'],cdsm['head_pos']=cdsm['fivegram_pos'].str.split(r'\s+').str
            except ValueError:
                phrases=pd.DataFrame()
                compounds=pd.DataFrame()
                modifiers=pd.DataFrame()
                heads=pd.DataFrame()
                return phrases,compounds,modifiers,heads
            
            phrases=relemjoin(phrases,'modifier_pos',lemmatize=False)
            phrases=relemjoin(phrases,'head_pos')
            cdsm=relemjoin(cdsm,'modifier_pos',lemmatize=False)
            cdsm=relemjoin(cdsm,'head_pos')
            
            phrases=pd.melt(phrases,id_vars=['modifier','head','decade','count'],value_vars=['l1_pos','l2_pos','l3_pos'])
            compounds=pd.melt(cdsm,id_vars=['modifier','head','decade','count'],value_vars=['l1_pos','l2_pos','l3_pos'])
            modifiers=pd.melt(cdsm,id_vars=['modifier','decade','count'],value_vars=['head','l1_pos','l2_pos','l3_pos'])
            heads=pd.melt(cdsm,id_vars=['head','decade','count'],value_vars=['modifier','l1_pos','l2_pos','l3_pos'])
            return phrases,compounds,modifiers,heads

In [56]:
def context_reducer(df):
    if len(df)==0:
        return df
    df["variable"]=df["variable"].str.replace(r"_pos","")
    df["context"],df["context_pos"]=df['value'].str.split('_', 1).str
    df.replace(spelling_replacement,inplace=True)
    df=df.loc[df.context_pos.isin(["noun","adj","adv","verb"])]
    df.replace(adv_replacement,inplace=True)
    df['context_pos']=df['context_pos'].str[0]
    if len(df)==0:
        return df
    df['context']=np.vectorize(lemma_maker)(df['context'], df['context_pos'])
    df['context']=df['context']+"_"+df['context_pos']
    df.query('context in @contextwords',inplace=True)
    #df.reset_index(inplace=True)
    return df

In [57]:
def cdsm_word_reducer(df):
    rightgram=syntactic_reducer(df,align="right",level="word")
    rightgram=context_reducer(rightgram)
    
    mid1gram=syntactic_reducer(df,align="mid1",level="word")
    mid1gram=context_reducer(mid1gram)
    
    mid2gram=syntactic_reducer(df,align="mid2",level="word")
    mid2gram=context_reducer(mid2gram)
    
    mid3gram=syntactic_reducer(df,align="mid3",level="word")
    mid3gram=context_reducer(mid3gram)
    
    leftgram=syntactic_reducer(df,align="left",level="word")
    leftgram=context_reducer(leftgram)    
    
    words_df=pd.concat([rightgram,mid1gram,mid2gram,mid3gram,leftgram],ignore_index=True)
    words_df.dropna(inplace=True)
    words_df=words_df.query('word in @words_list')
    words_df=words_df.groupby(['word','context','decade'])['count'].sum().to_frame()
    words_df.reset_index(inplace=True)
    words_df.decade=words_df.decade.astype("int32")
    return words_df

In [58]:
def cdsm_reducer(df):
    phrase_rightgram,compound_rightgram,modifier_rightgram,head_rightgram=syntactic_reducer(df,align="right")
    phrase_rightgram=context_reducer(phrase_rightgram)
    compound_rightgram=context_reducer(compound_rightgram)
    modifier_rightgram=context_reducer(modifier_rightgram)
    head_rightgram=context_reducer(head_rightgram)


    phrase_mid1gram,compound_mid1gram,modifier_mid1gram,head_mid1gram=syntactic_reducer(df,align="mid1")
    phrase_mid1gram=context_reducer(phrase_mid1gram)
    compound_mid1gram=context_reducer(compound_mid1gram)
    modifier_mid1gram=context_reducer(modifier_mid1gram)
    head_mid1gram=context_reducer(head_mid1gram)
 

    phrase_mid2gram,compound_mid2gram,modifier_mid2gram,head_mid2gram=syntactic_reducer(df,align="mid2")
    phrase_mid2gram=context_reducer(phrase_mid2gram)
    compound_mid2gram=context_reducer(compound_mid2gram)
    modifier_mid2gram=context_reducer(modifier_mid2gram)
    head_mid2gram=context_reducer(head_mid2gram)
    
    phrase_leftgram,compound_leftgram,modifier_leftgram,head_leftgram=syntactic_reducer(df,align="left")
    phrase_leftgram=context_reducer(phrase_leftgram)
    compound_leftgram=context_reducer(compound_leftgram)
    modifier_leftgram=context_reducer(modifier_leftgram)
    head_leftgram=context_reducer(head_leftgram)
    
    compounds=pd.concat([compound_rightgram,compound_mid1gram,compound_mid2gram,compound_leftgram],ignore_index=True)
    modifiers=pd.concat([modifier_rightgram,modifier_mid1gram,modifier_mid2gram,modifier_leftgram],ignore_index=True)
    heads=pd.concat([head_rightgram,head_mid1gram,head_mid2gram,head_leftgram],ignore_index=True)

    phrases=pd.concat([phrase_rightgram,phrase_mid1gram,phrase_mid2gram,phrase_leftgram],ignore_index=True)
    phrases.dropna(inplace=True)
    phrases=phrases.groupby(['modifier','head','context','decade'])['count'].sum().to_frame()
    phrases.reset_index(inplace=True)
    phrases.decade=phrases.decade.astype("int32")
    
    compounds.dropna(inplace=True)
    compounds=compounds.groupby(['modifier','head','context','decade'])['count'].sum().to_frame()
    compounds.reset_index(inplace=True)
    compounds.decade=compounds.decade.astype("int32")
    
    modifiers.dropna(inplace=True)
    modifiers=modifiers.groupby(['modifier','context','decade'])['count'].sum().to_frame()
    modifiers.reset_index(inplace=True)
    modifiers.decade=modifiers.decade.astype("int32")
    
    heads.dropna(inplace=True)
    heads=heads.groupby(['head','context','decade'])['count'].sum().to_frame()
    heads.reset_index(inplace=True)
    heads.decade=heads.decade.astype("int32")
    return compounds,modifiers,heads,phrases

In [59]:
def parallelize_dataframe(df, context_type="independant_word",num_cores = 70):
    num_partitions = num_cores
    df_split = np.array_split(df, num_partitions)
    print("Done splitting the datasets")
    pool = Pool(num_cores)

    cur_time=time.time()
    print("Starting parallelizing")
    if context_type=="dependant":

        results=pool.map_async(cdsm_reducer,df_split)
        pool.close()
        pool.join()

        results=results.get()

        
        print("Done parallelizing")
        print("Total time taken",round(time.time()-cur_time),"secs")
        compound_list = [ result[0] for result in results]
        compounds=pd.concat(compound_list,ignore_index=True)
        compounds=compounds.groupby(['modifier','head','context','decade'])['count'].sum().to_frame()
        compounds.reset_index(inplace=True)
        
        if not isfile("/data/dharp/compounding/datasets/compounds.csv"):
            compounds.to_csv("/data/dharp/compounding/datasets/compounds.csv",sep="\t",index=False)
        else:
            compounds.to_csv("/data/dharp/compounding/datasets/compounds.csv", mode='a',sep="\t", header=False,index=False)
        
        
        modifier_list = [ result[1] for result in results]
        modifiers=pd.concat(modifier_list,ignore_index=True)
        modifiers=modifiers.groupby(['modifier','context','decade'])['count'].sum().to_frame()
        modifiers.reset_index(inplace=True)

        if not isfile("/data/dharp/compounding/datasets/modifiers.csv"):
            modifiers.to_csv("/data/dharp/compounding/datasets/modifiers.csv",sep="\t",index=False)
        else:
            modifiers.to_csv("/data/dharp/compounding/datasets/modifiers.csv", mode='a', header=False,index=False)
        
        head_list = [ result[2] for result in results]
        heads=pd.concat(head_list,ignore_index=True)
        heads=heads.groupby(['head','context','decade'])['count'].sum().to_frame()
        heads.reset_index(inplace=True)

        if not isfile("/data/dharp/compounding/datasets/heads.csv"):
            heads.to_csv("/data/dharp/compounding/datasets/heads.csv",sep="\t",index=False)
        else:
            heads.to_csv("/data/dharp/compounding/datasets/heads.csv", mode='a', header=False,index=False)
            
        phrase_list = [ result[3] for result in results]
        phrases=pd.concat(phrase_list,ignore_index=True)
        phrases=phrases.groupby(['modifier','head','context','decade'])['count'].sum().to_frame()
        phrases.reset_index(inplace=True)
        
        if not isfile("/data/dharp/compounding/datasets/phrases.csv"):
            phrases.to_csv("/data/dharp/compounding/datasets/phrases.csv",sep="\t",index=False)
        else:
            phrases.to_csv("/data/dharp/compounding/datasets/phrases.csv", mode='a', header=False,index=False)

    elif context_type=="independant_word":
        words_list=[]
        results=pool.map_async(cdsm_word_reducer,df_split)
  
        
        pool.close()
        pool.join()
        print("Done parallelizing")
        print("Total time taken",round(time.time()-cur_time),"secs")
        word_list=results.get()
        words = pd.concat(words_list,ignore_index=True)
        words=words.groupby(['word','context','decade'])['count'].sum().to_frame()
        words.reset_index(inplace=True)
        
                
        if not isfile("/data/dharp/compounding/datasets/words.csv"):
            words.to_csv("/data/dharp/compounding/datasets/words.csv",sep="\t",index=False)
        else:
            words.to_csv("/data/dharp/compounding/datasets/words.csv", mode='a',sep="\t", header=False,index=False)
        
    print("Done concatenations \n")

In [60]:
def pkl_reader(files,size=None):
    cur_time=time.time()
    print("Reading the pickle files")
    pkl_list=[]
    if size is None:
        for f in files:
            temp_df=pd.read_pickle(f)
            if temp_df.empty==True:
                print("File ",f," is empty")
                continue
            pkl_list.append(temp_df)
    else:
        total_size=0
        for f in files:
            if total_size>size:
                break
            temp_df=pd.read_pickle(f)
            if temp_df.empty==True:
                print("File ",f," is empty")
                continue
            total_size+=temp_df.shape[0]
            pkl_list.append(temp_df)
    print("Done reading the files \n")
    entire_df=pd.concat(pkl_list,ignore_index=True)
    print("Done concatenating the datasets \n")
    print("Total time taken",round(time.time()-cur_time),"secs \n")
    return entire_df.head(size)

In [61]:
def split_and_assign(df,split_col,splitter,col_names):
    temp_cols=df[split_col].str.split(splitter,len(col_names)).str
    df = df.assign(**{k:temp_cols[i] for i,k in enumerate(col_names)})
    #df.drop(split_col,axis=1,inplace=True)
    return df

In [62]:
num_cores=8
f = '/data/dharp/compounding/datasets/entire_df_0_.h5'
print(f,'is read in')
tmp_df=pd.read_hdf(f, key='df')
tmp_df=tmp_df.head(10_000_000)
#print("Compounds and Phrases")
parallelize_dataframe(tmp_df,context_type="independant_word",num_cores=num_cores)
print("Done writing to files")

/data/dharp/compounding/datasets/entire_df_0_.h5 is read in
Done splitting the datasets
Starting parallelizing


Process ForkPoolWorker-242:
Process ForkPoolWorker-237:
Traceback (most recent call last):
Process ForkPoolWorker-243:
Traceback (most recent call last):
Process ForkPoolWorker-239:
  File "/data/dharp/packages/miniconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/data/dharp/packages/miniconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Process ForkPoolWorker-244:
  File "/data/dharp/packages/miniconda3/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/data/dharp/packages/miniconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/data/dharp/packages/miniconda3/lib/python3.6/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "/data/dharp/packages/miniconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **s

KeyboardInterrupt: 

  File "/data/dharp/packages/miniconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/data/dharp/packages/miniconda3/lib/python3.6/site-packages/pandas/core/internals.py", line 3754, in replace_list
    mgr=mgr, convert=convert)
  File "<ipython-input-57-dbb213186c1d>", line 3, in cdsm_word_reducer
    rightgram=context_reducer(rightgram)
  File "/data/dharp/packages/miniconda3/lib/python3.6/site-packages/pandas/core/internals.py", line 3751, in replace_list
    if b.dtype == np.object_:
  File "/data/dharp/packages/miniconda3/lib/python3.6/site-packages/pandas/core/frame.py", line 3798, in replace
    method=method)
  File "/data/dharp/packages/miniconda3/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/data/dharp/packages/miniconda3/lib/python3.6/site-packages/pandas/core/internals.py", line 2440, in replace
    convert=convert, mgr=mgr)
  File "/data/dharp

  File "/data/dharp/packages/miniconda3/lib/python3.6/site-packages/pandas/core/internals.py", line 802, in replace
    blocks = self.putmask(mask, value, inplace=inplace)
  File "/data/dharp/packages/miniconda3/lib/python3.6/site-packages/pandas/core/internals.py", line 969, in putmask
    new_values = self.values if inplace else self.values.copy()
KeyboardInterrupt
Traceback (most recent call last):
  File "/data/dharp/packages/miniconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/data/dharp/packages/miniconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/data/dharp/packages/miniconda3/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/data/dharp/packages/miniconda3/lib/python3.6/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "<ipython-input-57-dbb213186c1d>", line 3, in cdsm_w

In [ ]:
cdsm_word_reducer(tmp_df)